In [0]:
%sql
WITH date_diffs AS (
    SELECT 
        user_id,
        purchase_date,
        LAG(purchase_date, 1) OVER (PARTITION BY user_id ORDER BY purchase_date) AS prev_date_1,
        LAG(purchase_date, 2) OVER (PARTITION BY user_id ORDER BY purchase_date) AS prev_date_2
    FROM (
        SELECT DISTINCT user_id, purchase_date  -- Remove duplicates on same day
        FROM purchases
    ) distinct_purchases
)
SELECT DISTINCT user_id
FROM date_diffs
WHERE 
    DATEDIFF(day, prev_date_2, prev_date_1) = 1
    AND DATEDIFF(day, prev_date_1, purchase_date) = 1;


In [0]:
%sql
WITH monthly_revenue AS (
    SELECT
        product_category,
        DATE_TRUNC('month', sale_date) AS month,  -- or DATE_FORMAT depending on your SQL dialect
        SUM(revenue) AS monthly_revenue
    FROM sales
    GROUP BY product_category, DATE_TRUNC('month', sale_date)
),
with_previous AS (
    SELECT
        product_category,
        month,
        monthly_revenue,
        LAG(monthly_revenue, 1) OVER (
            PARTITION BY product_category 
            ORDER BY month
        ) AS prev_month_revenue
    FROM monthly_revenue
)
SELECT
    product_category,
    month,
    monthly_revenue,
    prev_month_revenue,
    ROUND(
        ((monthly_revenue - prev_month_revenue) / prev_month_revenue) * 100, 
        2
    ) AS percent_change
FROM with_previous
WHERE prev_month_revenue IS NOT NULL  -- Exclude first month (no previous data)
ORDER BY product_category, month;



In [0]:
%sql
WITH depo_sal AS (
  SELECT
  department_id, AVG(e.salary) AS avg_salary
  FROM employees
  GROUP BY department_id
)

SELECT e.name, e.department_name, e.salary, ds.avg_salary
FROM employees e
INNER JOIN departments d
ON e.department_id = d.department_id
INNER JOIN depo_sal ds
ON e.department_id = ds.department_id
WHERE e.salary > ds.avg_salary
ORDER BY e.salary DESC


In [0]:
import pandas as pd

transactions = [
    {'customer_id': 1, 'amount': 100, 'email': 'test@email.com'},
    {'customer_id': 1, 'amount': 150, 'email': 'test\email.com'},
    { 'amount': 200, 'email': 'test@email.com'},
    # ... more transactions
]

t1 = pd.DataFrame(transactions)
pattern = r"^[\w\.-]+@[\w\.-]+\.\w+$"
print(~t1['email'].str.contains(pattern))

"""**Q22.** Implement a data validation function that checks if a pandas DataFrame meets these requirements:
# - No duplicate rows
# - Column 'email' contains valid email format
# - Column 'age' is between 0 and 120
# - No missing values in 'user_id'

Return a dictionary with validation results.
"""

def transactions_validation(df):
    val_dict = {'Duplicated Rows': False, 'Invalid Emails': False, 'Age out of range': False, 'Missing User Id': False}
    cleaned_df = df.copy()

    if df!=df[~df.duplicated()]:
        val_dict['Duplicated Rows'] = True
        cleaned_df = cleaned_df.drop_duplicates()
    
    pattern = r"^[\w\.-]+@[\w\.-]+\.\w+$"
    if ~cleaned_df['email'].str.contains(pattern).any()==True:
        val_dict['Invalid Email'] = True
        cleaned_df = cleaned_df[cleaned_df['email'].str.contains(pattern)]

    age_valid = (cleaned_df['age'] >= 0) & (cleaned_df['age'] <= 120)
    if (~age_valid).any():
        val_dict['Age out of range'] = True
        cleaned_df = cleaned_df[age_valid]
    
    if df['user_id'].isna().any():
        val_dict['Missing User Id'] = True
        cleaned_df = cleaned_df[~cleaned_df['user_id'].isna()]
    
    return val_dict, cleaned_df

"""
**Q23.** Write a Python function that connects to a database, executes a query in chunks to avoid memory issues, and writes results to a CSV file incrementally. Use proper error handling and context managers.
"""